In [1]:
import sys
sys.path.append("..")

In [2]:
import pandas as pd
import numpy as np
from reaction import get_reaction_center, Reaction, single_step_synthesis, convert_product_and_reactant, single_step_synthesis_v2
import os
from rdkit import Chem
from create_data import clean_mapped_num, extract_mapped_reaction, save_reaction_center
from data import make_useful_data

/home/stein/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 自定义数据位置

In [11]:
data_folder_path = "final_csv/" 

### 提取反应中心

In [5]:
mapped_reaction_list, reaction_smiles_list = extract_mapped_reaction(file_path=data_folder_path+'/'+"first_mapped_reaction.smi")
print(len(mapped_reaction_list))
dataframe = save_reaction_center(mapped_reaction_list, save_csv=False)

216965


### 清理map num

In [6]:
reaction_center_radius_smiles0 =  dataframe['reaction_center_radius_smiles0'] #radius为0
reaction_center_radius_smiles1 = dataframe['reaction_center_radius_smiles1'] #radius为1
reaction_center_radius_smiles2 = dataframe['reaction_center_radius_smiles2'] #radius为2
reaction_center_smiles_list = dataframe["reaction_center_smiles"]

#定义list用来储存clean过之后的数据
clean_reaction_center_radius_smiles0 = []
clean_reaction_center_radius_smarts0 = []
clean_reaction_center_radius_smiles1 = []
clean_reaction_center_radius_smarts1 = []
clean_reaction_center_radius_smiles2 = []
clean_reaction_center_radius_smarts2 = []

for index, synthesis_smiles in enumerate(reaction_center_radius_smiles0):
    if index % 100 == 0:
        print (index)
    clean_reaction_rule_radius_smiles, clean_reaction_rule_radius_smarts = clean_mapped_num(synthesis_smiles)
    clean_reaction_center_radius_smiles0.append(clean_reaction_rule_radius_smiles)
    clean_reaction_center_radius_smarts0.append(clean_reaction_rule_radius_smarts)

for index, synthesis_smiles in enumerate(reaction_center_radius_smiles1):
    if index % 100 == 0:
        print (index)
    clean_reaction_rule_radius_smiles, clean_reaction_rule_radius_smarts = clean_mapped_num(synthesis_smiles)
    clean_reaction_center_radius_smiles1.append(clean_reaction_rule_radius_smiles)
    clean_reaction_center_radius_smarts1.append(clean_reaction_rule_radius_smarts)

for index, synthesis_smiles in enumerate(reaction_center_radius_smiles2):
    if index % 100 == 0:
        print (index)
    clean_reaction_rule_radius_smiles, clean_reaction_rule_radius_smarts = clean_mapped_num(synthesis_smiles)
    clean_reaction_center_radius_smiles2.append(clean_reaction_rule_radius_smiles)
    clean_reaction_center_radius_smarts2.append(clean_reaction_rule_radius_smarts)


clean_reaction_center_smiles = [] 
clean_reaction_center_smarts = []
for index, synthesis_smiles in enumerate(reaction_center_smiles_list):
    if index % 100 == 0:
        print (index)
    clean_reaction_rule_smiles, clean_reaction_rule_smarts = clean_mapped_num(synthesis_smiles)
    clean_reaction_center_smiles.append(clean_reaction_rule_smiles)
    clean_reaction_center_smarts.append(clean_reaction_rule_smarts)
dataframe['clean_reaction_center_radius_smiles0'] = clean_reaction_center_radius_smiles0
dataframe['clean_reaction_center_radius_smiles1'] = clean_reaction_center_radius_smiles1
dataframe['clean_reaction_center_radius_smiles2'] = clean_reaction_center_radius_smiles2
dataframe["clean_reaction_center_radius_smarts0"] = clean_reaction_center_radius_smarts0
dataframe["clean_reaction_center_radius_smarts1"] = clean_reaction_center_radius_smarts1
dataframe["clean_reaction_center_radius_smarts2"] = clean_reaction_center_radius_smarts2
print("clean_reaction_center_radius_smarts2",clean_reaction_center_radius_smarts2)

dataframe['clean_reaction_center_smiles'] = clean_reaction_center_smiles
dataframe['clean_reaction_center_smarts'] = clean_reaction_center_smarts

NameError: name 'dataframe' is not defined

### 我们仅仅使用 smarts 信息
#### expandnetwork 的数据用的是radius为0时的数据
#### rolloutnetwork 的数据用的是radius为1时的数据

#### 提取出现过occurance_threshold次的反应（原文 rollout network 的出现阀值是50）

In [ ]:
def rule_filter(labels, occurance_threshold = 2):
    count = np.bincount(labels)
    coverage_rate = count[count>=occurance_threshold].sum()/len(labels)
    print("coverage_rate", coverage_rate)
    keeped_labels = np.where(count>=occurance_threshold)[0]
    the_filter = np.array([labels[i] in keeped_labels for i in range(len(labels))])
    return the_filter

In [ ]:
def clean_data(data):
    smart1_filter = []
    smart2_filter = []
    data.reset_index(drop=True, inplace=True)
    smarts1 = data.clean_reaction_center_radius_smarts0.values
    smarts2 = data.clean_reaction_center_radius_smarts1.values
    for i in range(len(smarts1)):
        if smarts1[i][:2] == ">>":
            smart1_filter.append(False)
        else:
            smart1_filter.append(True)
        if smarts2[i][:2] == ">>":
            smart2_filter.append(False)
        else:
            smart2_filter.append(True)
    smart1_filter = np.array(smart1_filter)
    smart2_filter = np.array(smart2_filter)
    new_data = data.iloc[np.logical_and(smart1_filter, smart2_filter)]
    new_data.reset_index(drop=True, inplace=True)
    return new_data

In [ ]:
def make_useful_data(data, save_path):
    data = clean_data(data)
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    data.reset_index(drop=True, inplace=True)
    lbl0 = LabelEncoder()
    lbl1 = LabelEncoder()
    lbl_exp = LabelEncoder()
    lbl_roll = LabelEncoder()
    expand_rule_labels = lbl0.fit_transform(data.clean_reaction_center_radius_smarts0.values)
    rollout_rule_labels = lbl1.fit_transform(data.clean_reaction_center_radius_smarts1.values)
    expand_rule_filter = rule_filter(labels=expand_rule_labels, occurance_threshold=2)
    rollout_rule_filter = rule_filter(labels=rollout_rule_labels, occurance_threshold=3)
    data_expand_network = data[expand_rule_filter]
    data_expand_network.reset_index(drop=True, inplace=True)
    oexpand_rule_labels = lbl0.inverse_transform(expand_rule_labels[expand_rule_filter])
    lbl_exp.fit_transform(oexpand_rule_labels)
    data_expand_network["y"] = lbl_exp.fit_transform(oexpand_rule_labels)
    del data_expand_network["clean_reaction_center_radius_smarts1"]  
    data_expand_network.columns = ["mapped_reaction_smiles", "reaction_center_radius0", 'y']

    data_rollout_network = data[rollout_rule_filter]
    data_rollout_network.reset_index(drop=True, inplace=True)
    orollout_rule_labels = lbl1.inverse_transform(rollout_rule_labels[rollout_rule_filter])
    data_rollout_network["y"] = lbl_roll.fit_transform(orollout_rule_labels)
    del data_rollout_network["clean_reaction_center_radius_smarts0"]
    data_rollout_network.columns = ["mapped_reaction_smiles", "reaction_center_radius1", 'y']

    data_expand_network.to_csv(os.path.join(save_path, "data_expand_network.csv"), index=False)
    data_rollout_network.to_csv(os.path.join(save_path, "data_rollout_network.csv"), index = False)

    np.save(os.path.join(save_path, "expand_rule_label_encoder.npy"), lbl_exp.classes_)
    np.save(os.path.join(save_path, "rollout_rule_label_encoder.npy"), lbl_roll.classes_)

In [ ]:
dataframe = dataframe["mapped_reaction_smiles","clean_reaction_center_radius_smarts0","clean_reaction_center_radius_smarts1"]

In [ ]:
dataframe.reset_index(drop=True, inplace = True)

In [ ]:
make_useful_data(data=data, save_path="useful_data")